In [93]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd

## Travis County Boundary API

In [94]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [95]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

## Region Mapping

In [96]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [97]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 3200
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [98]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
longitude_delta

0.013847870403828138

In [99]:
lat_lon = zip(latitudes, longitudes)

In [100]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [101]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [102]:
map_point_to_region(30.084446, -97.702885)

36

In [103]:
get_representative(13)

[-98.02650084954831, 30.08076594002934]

In [104]:
valid_regions = set()
for lat, lon in zip(latitudes, longitudes):
    valid_regions.add(map_point_to_region(lat, lon))

In [105]:
valid_regions = list(valid_regions)
valid_regions.sort()

In [106]:
region_coords = [get_representative(i) for i in range(3200)]
pprint.pprint(region_coords)

[[-98.20652316479809, 30.08076594002934],
 [-98.19267529439426, 30.08076594002934],
 [-98.17882742399043, 30.08076594002934],
 [-98.1649795535866, 30.08076594002934],
 [-98.15113168318277, 30.08076594002934],
 [-98.13728381277895, 30.08076594002934],
 [-98.12343594237511, 30.08076594002934],
 [-98.10958807197129, 30.08076594002934],
 [-98.09574020156747, 30.08076594002934],
 [-98.08189233116363, 30.08076594002934],
 [-98.06804446075981, 30.08076594002934],
 [-98.05419659035597, 30.08076594002934],
 [-98.04034871995215, 30.08076594002934],
 [-98.02650084954831, 30.08076594002934],
 [-98.01265297914449, 30.08076594002934],
 [-97.99880510874067, 30.08076594002934],
 [-97.98495723833683, 30.08076594002934],
 [-97.97110936793301, 30.08076594002934],
 [-97.95726149752917, 30.08076594002934],
 [-97.94341362712535, 30.08076594002934],
 [-97.92956575672152, 30.08076594002934],
 [-97.9157178863177, 30.08076594002934],
 [-97.90187001591387, 30.08076594002934],
 [-97.88802214551004, 30.08076594002

 [-97.44489029258754, 30.343755061437694],
 [-97.43104242218371, 30.343755061437694],
 [-97.41719455177989, 30.343755061437694],
 [-97.40334668137605, 30.343755061437694],
 [-97.38949881097223, 30.343755061437694],
 [-97.37565094056839, 30.343755061437694],
 [-98.20652316479809, 30.354712941496377],
 [-98.19267529439426, 30.354712941496377],
 [-98.17882742399043, 30.354712941496377],
 [-98.1649795535866, 30.354712941496377],
 [-98.15113168318277, 30.354712941496377],
 [-98.13728381277895, 30.354712941496377],
 [-98.12343594237511, 30.354712941496377],
 [-98.10958807197129, 30.354712941496377],
 [-98.09574020156747, 30.354712941496377],
 [-98.08189233116363, 30.354712941496377],
 [-98.06804446075981, 30.354712941496377],
 [-98.05419659035597, 30.354712941496377],
 [-98.04034871995215, 30.354712941496377],
 [-98.02650084954831, 30.354712941496377],
 [-98.01265297914449, 30.354712941496377],
 [-97.99880510874067, 30.354712941496377],
 [-97.98495723833683, 30.354712941496377],
 [-97.971109

 [-97.95726149752917, 30.53003902243528],
 [-97.94341362712535, 30.53003902243528],
 [-97.92956575672152, 30.53003902243528],
 [-97.9157178863177, 30.53003902243528],
 [-97.90187001591387, 30.53003902243528],
 [-97.88802214551004, 30.53003902243528],
 [-97.87417427510621, 30.53003902243528],
 [-97.86032640470238, 30.53003902243528],
 [-97.84647853429856, 30.53003902243528],
 [-97.83263066389472, 30.53003902243528],
 [-97.8187827934909, 30.53003902243528],
 [-97.80493492308707, 30.53003902243528],
 [-97.79108705268324, 30.53003902243528],
 [-97.77723918227942, 30.53003902243528],
 [-97.76339131187558, 30.53003902243528],
 [-97.74954344147176, 30.53003902243528],
 [-97.73569557106792, 30.53003902243528],
 [-97.7218477006641, 30.53003902243528],
 [-97.70799983026028, 30.53003902243528],
 [-97.69415195985644, 30.53003902243528],
 [-97.68030408945262, 30.53003902243528],
 [-97.66645621904878, 30.53003902243528],
 [-97.65260834864496, 30.53003902243528],
 [-97.63876047824112, 30.530039022435

## Batch calls of distance matrix

In [107]:
# Prepare data for api call

locations = region_coords
# locations = region_coords + ems_coords + hospital_coords
durations = [[0 for i in range(3200)] for j in range(3200)]

#pprint.pprint(locations)
print(len(locations))

3200


In [91]:
batch_size = 1
num_batches = math.ceil(len(valid_regions) / batch_size)
num_batches_per_round = math.ceil(num_batches/4)
current_branch = 3
remainder = len(valid_regions) % batch_size
print(num_batches)

1948


In [92]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
#     'Authorization': '5b3ce3597851110001cf62485eef3a0ab19d4b3e8ad2d8c88dca3d19',
    'Content-Type': 'application/json; charset=utf-8'
}

In [87]:
for i in range(num_batches_per_round*current_branch, min(num_batches_per_round*current_branch+num_batches_per_round, len(valid_regions))):
    print("batch ", i)
    start = i * batch_size
    sources = list(range(start, start + batch_size))

    # handle last partial batch
    if(sources[batch_size - 1] >= len(locations)):
        sources = sources[:remainder]

    locations = [get_representative(valid_regions[j]) for j in range(len(valid_regions))]

    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

    duration_batch = call.json()['durations']

    for j in range(len(duration_batch)):
        for k in range(len(duration_batch[0])):
            durations[valid_regions[i*batch_size + j]][valid_regions[k]] = duration_batch[j][k]
    time.sleep(0.1)


batch  1461
batch  1462
batch  1463
batch  1464
batch  1465
batch  1466
batch  1467
batch  1468
batch  1469
batch  1470
batch  1471
batch  1472
batch  1473


KeyError: 'durations'

In [43]:
len(valid_regions)/490

3.975510204081633

In [30]:
print(len(durations), len(durations[0]))

467 467


In [78]:
np.save("durations4.npy", durations)

In [108]:
durations = np.load("durations3.npy", allow_pickle=True)

In [ ]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d["time_matrix"] = durations
with open("grid_info.json", "w") as j:
    json.dump(d, j)